In [10]:
# bad solution for part 1
import sys
import functools

class Graph:
    def __init__(self, nodes, init_graph):
        self.nodes = nodes
        self.graph = self.construct_graph(nodes, init_graph)
        
    def construct_graph(self, nodes, init_graph):
        graph = {}
        for node in nodes:
            graph[node] = {}
        
        graph.update(init_graph)
                   
        return graph
    
    def get_nodes(self):
        return self.nodes
    
    def get_outgoing_edges(self, node):
        connections = []
        for out_node in self.nodes:
            if self.graph[node].get(out_node, False) != False:
                connections.append(out_node)
        return connections
    
    def value(self, node1, node2):
        return self.graph[node1][node2]

nodes = []
relations = {}
global flowRates
flowRates = {}
global PLSBECORRECT 
PLSBECORRECT = -99999

with open("data.txt") as f:
    for line in f:
        line = line.strip().split(" ")
        valve = line[1]
        rate = int(line[4][5:-1])
        connections = [x.strip(", ") for x in line[9:]]
        
        flowRates[valve] = rate
        nodes.append(valve)
        
        relations[valve] = {}
        for i in connections:
            relations[valve][i] = 1

global map_        
map_ = Graph(nodes, relations)

@functools.lru_cache(maxsize=None)
def move(timeLeft, totalPressure, curNode, openValves):
    if timeLeft <= 0:
        global PLSBECORRECT
        if totalPressure > PLSBECORRECT:
            PLSBECORRECT = totalPressure
            
        return 0
    
    elif flowRates[curNode] > 0 and not curNode in openValves.split(" "):
        move(timeLeft-1, totalPressure + ((timeLeft-1) * flowRates[curNode]), curNode, openValves + f" {curNode}")
        
    else:
        for valve in map_.get_outgoing_edges(curNode):
            move(timeLeft-1, totalPressure, valve, openValves)
        
    return 0
        
    
move(30, 0, "AA", "")
print(PLSBECORRECT)

2056


In [3]:
# better solution for part 1
import sys
import functools

class Graph:
    def __init__(self, nodes, init_graph):
        self.nodes = nodes
        self.graph = self.construct_graph(nodes, init_graph)
        
    def construct_graph(self, nodes, init_graph):
        graph = {}
        for node in nodes:
            graph[node] = {}
        
        graph.update(init_graph)
                   
        return graph
    
    def get_nodes(self):
        return self.nodes
    
    def get_outgoing_edges(self, node):
        connections = []
        for out_node in self.nodes:
            if self.graph[node].get(out_node, False) != False:
                connections.append(out_node)
        return connections
    
    def value(self, node1, node2):
        return self.graph[node1][node2]
    
def dijkstra_algorithm(graph, start_node):
    unvisited_nodes = list(graph.get_nodes())
    
    shortest_path = {}
 
    previous_nodes = {}
   
    max_value = sys.maxsize
    for node in unvisited_nodes:
        shortest_path[node] = max_value
        
    shortest_path[start_node] = 0
    
    while unvisited_nodes:
        current_min_node = None
        for node in unvisited_nodes:
            if current_min_node == None:
                current_min_node = node
            elif shortest_path[node] < shortest_path[current_min_node]:
                current_min_node = node
                
        neighbors = graph.get_outgoing_edges(current_min_node)
        for neighbor in neighbors:
            tentative_value = shortest_path[current_min_node] + graph.value(current_min_node, neighbor)
            if tentative_value < shortest_path[neighbor]:
                shortest_path[neighbor] = tentative_value
                previous_nodes[neighbor] = current_min_node
 
        unvisited_nodes.remove(current_min_node)
    
    return shortest_path

nodes = []
relations = {}
flowRates = {}
pairwiseDistance = {}

with open("data.txt") as f:
    for line in f:
        line = line.strip().split(" ")
        valve = line[1]
        rate = int(line[4][5:-1])
        connections = [x.strip(", ") for x in line[9:]]
        
        if rate > 0:
            flowRates[valve] = rate
        nodes.append(valve)
        
        relations[valve] = {}
        for i in connections:
            relations[valve][i] = 1

map_ = Graph(nodes, relations)

for i in nodes:
    pairwiseDistance[i] = {}
    shortest_path = dijkstra_algorithm(map_, i)
    for j in nodes:
        if j != i:
            pairwiseDistance[i][j] = shortest_path[j]

global paths
paths = []
            
def move(timeLeft, curNode, pairwiseDistance, flowRates, curPath):
    global paths
    if timeLeft <= 0:
        paths.append(curPath + f" {curNode}")
        return 0
    else:
        noAvaliablePath = True
        for valve in flowRates.keys():
            if curNode == valve or valve in curPath.split(" "):
                continue
            distance = pairwiseDistance[curNode][valve]
            if timeLeft-distance > 0:
                noAvaliablePath = False
                move(timeLeft-distance-1, valve, pairwiseDistance, flowRates, curPath + f" {curNode}")
        if noAvaliablePath:
            paths.append(curPath + f" {curNode}")
                
    return 0
move(30, "AA", pairwiseDistance, flowRates, "")
print(len(paths))
best = [-99999999, None]
for path in paths:
    timeLeft = 30
    total = 0
    path = path.strip(" ").split(" ")
    for node in range(1, len(path)):
        timeLeft -= pairwiseDistance[path[node-1]][path[node]] + 1
        total += flowRates[path[node]] * timeLeft
        
    if total > best[0]:
        best = [total, path]

print(best)

256400
[2056, ['AA', 'IR', 'LY', 'WL', 'CP', 'UC', 'SS', 'DD', 'EU']]


In [1]:
# part 2
import sys
import functools

class Graph:
    def __init__(self, nodes, init_graph):
        self.nodes = nodes
        self.graph = self.construct_graph(nodes, init_graph)
        
    def construct_graph(self, nodes, init_graph):
        graph = {}
        for node in nodes:
            graph[node] = {}
        
        graph.update(init_graph)
                   
        return graph
    
    def get_nodes(self):
        return self.nodes
    
    def get_outgoing_edges(self, node):
        connections = []
        for out_node in self.nodes:
            if self.graph[node].get(out_node, False) != False:
                connections.append(out_node)
        return connections
    
    def value(self, node1, node2):
        return self.graph[node1][node2]
    
def dijkstra_algorithm(graph, start_node):
    unvisited_nodes = list(graph.get_nodes())
    
    shortest_path = {}
 
    previous_nodes = {}
   
    max_value = sys.maxsize
    for node in unvisited_nodes:
        shortest_path[node] = max_value
        
    shortest_path[start_node] = 0
    
    while unvisited_nodes:
        current_min_node = None
        for node in unvisited_nodes:
            if current_min_node == None:
                current_min_node = node
            elif shortest_path[node] < shortest_path[current_min_node]:
                current_min_node = node
                
        neighbors = graph.get_outgoing_edges(current_min_node)
        for neighbor in neighbors:
            tentative_value = shortest_path[current_min_node] + graph.value(current_min_node, neighbor)
            if tentative_value < shortest_path[neighbor]:
                shortest_path[neighbor] = tentative_value
                previous_nodes[neighbor] = current_min_node
 
        unvisited_nodes.remove(current_min_node)
    
    return shortest_path

nodes = []
relations = {}
flowRates = {}
pairwiseDistance = {}

with open("data.txt") as f:
    for line in f:
        line = line.strip().split(" ")
        valve = line[1]
        rate = int(line[4][5:-1])
        connections = [x.strip(", ") for x in line[9:]]
        
        if rate > 0:
            flowRates[valve] = rate
        nodes.append(valve)
        
        relations[valve] = {}
        for i in connections:
            relations[valve][i] = 1

map_ = Graph(nodes, relations)

for i in nodes:
    pairwiseDistance[i] = {}
    shortest_path = dijkstra_algorithm(map_, i)
    for j in nodes:
        if j != i:
            pairwiseDistance[i][j] = shortest_path[j]

    
global paths
paths = []
global elephantPaths
            
def move(timeLeft, curNode, pairwiseDistance, flowRates, curPath):
    global paths
    if timeLeft <= 0:
        paths.append(curPath + f" {curNode}")
        return 0
    else:
        noAvaliablePath = True
        for valve in flowRates.keys():
            if curNode == valve or valve in curPath.split(" "):
                continue
            distance = pairwiseDistance[curNode][valve]
            if timeLeft-distance > 0:
                noAvaliablePath = False
                move(timeLeft-distance-1, valve, pairwiseDistance, flowRates, curPath + f" {curNode}")
        if noAvaliablePath:
            paths.append(curPath + f" {curNode}")
                
    return 0

def moveElephant(timeLeft, curNode, pairwiseDistance, flowRates, curPath):
    global elephantPaths
    if timeLeft <= 0:
        elephantPaths.append(curPath + f" {curNode}")
        return 0
    else:
        noAvaliablePath = True
        for valve in flowRates.keys():
            if curNode == valve or valve in curPath.split(" "):
                continue
            distance = pairwiseDistance[curNode][valve]
            if timeLeft-distance > 0:
                noAvaliablePath = False
                moveElephant(timeLeft-distance-1, valve, pairwiseDistance, flowRates, curPath + f" {curNode}")
        if noAvaliablePath:
            elephantPaths.append(curPath + f" {curNode}")
                
    return 0

move(26, "AA", pairwiseDistance, flowRates, "")

best = [-99999999, None, None]
counter = 0
for x in paths:
    counter += 1
    if counter % 250 == 0:
        print(f"Calculating.. {counter}/{len(paths)}")
    
    timeLeft = 26
    total = 0
    path = x.strip(" ").split(" ")
    for node in range(1, len(path)):
        timeLeft -= pairwiseDistance[path[node-1]][path[node]] + 1
        total += flowRates[path[node]] * timeLeft
        
    #remove human valves
    elephantFlowRates = flowRates.copy()
    for i in path:
        if i != "AA":
            elephantFlowRates.pop(i)
            
    elephantPaths = []
    moveElephant(26, "AA", pairwiseDistance, elephantFlowRates, "")
    
    for y in elephantPaths:
        timeLeftElephant = 26
        totalElephant = 0
        pathElephant = y.strip(" ").split(" ")
        for nodeElephant in range(1, len(pathElephant)):
            timeLeftElephant -= pairwiseDistance[pathElephant[nodeElephant-1]][pathElephant[nodeElephant]] + 1
            totalElephant += flowRates[pathElephant[nodeElephant]] * timeLeftElephant
            
        if total + totalElephant > best[0]:
            best = [total + totalElephant, path , pathElephant]
    
        
print(best)

Calculating.. 250/59816
Calculating.. 500/59816
Calculating.. 750/59816
Calculating.. 1000/59816
Calculating.. 1250/59816
Calculating.. 1500/59816
Calculating.. 1750/59816
Calculating.. 2000/59816
Calculating.. 2250/59816
Calculating.. 2500/59816
Calculating.. 2750/59816
Calculating.. 3000/59816
Calculating.. 3250/59816
Calculating.. 3500/59816
Calculating.. 3750/59816
Calculating.. 4000/59816
Calculating.. 4250/59816
Calculating.. 4500/59816
Calculating.. 4750/59816
Calculating.. 5000/59816
Calculating.. 5250/59816
Calculating.. 5500/59816
Calculating.. 5750/59816
Calculating.. 6000/59816
Calculating.. 6250/59816
Calculating.. 6500/59816
Calculating.. 6750/59816
Calculating.. 7000/59816
Calculating.. 7250/59816
Calculating.. 7500/59816
Calculating.. 7750/59816
Calculating.. 8000/59816
Calculating.. 8250/59816
Calculating.. 8500/59816
Calculating.. 8750/59816
Calculating.. 9000/59816
Calculating.. 9250/59816
Calculating.. 9500/59816
Calculating.. 9750/59816
Calculating.. 10000/59816
Ca